# Tutorial 4: Generate context-aware gene embedding

In [ ]:
import torch.multiprocessing as mp
import pickle
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict, OrderedDict
from tqdm import tqdm
from argparse import Namespace
import gc
import csv

from model.constants import *
import os
import torch.utils.data as data
from copy import deepcopy
import random
from sklearn.preprocessing import LabelEncoder
from omics.constants import *
import h5py
from pytorch_lightning.callbacks import (EarlyStopping, LearningRateMonitor,
                                         ModelCheckpoint, Callback)
from math import exp
import random
import h5py
import hashlib
import anndata as ad
from model.emb_gen.pretrain import Omics
from pytorch_lightning import LightningModule, Trainer, seed_everything
from scipy.spatial import cKDTree
from omics.baseline.imputation.preprocess.spot_input_gen import load_spot_data_with_full_info
from step1_cell_specific_gene_emb_gen import process_single_gpu_gene_pairs

## Initialize args and fix seed

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 直接按照命令行脚本写死推理配置，避免 Notebook 再次解析 CLI
args = Namespace(
    ckpt_path="/media/dang/Omics/data/ckpts/Omics/cosmx_1_cosmx_lung5_rep1_2025_09_13_10_31_59/epoch=2-step=4131.ckpt",
    dataset_name="cosmx_lung5_rep1",
    target_dataset="cosmx_lung5_rep1",
    gene_pct=100,
    radius=20,
    max_points=20,
    cell_gene_pairs_path="/path/to/cell_gene_pairs.csv",  # TODO: 替换成实际的 cell-gene pair CSV
    linear_hidden_dim=256,
    config="/media/dang/Omics/omics/configs/bert_config_5-12.json",
    seed=42,
    f=None,
    split_slice=None,
    target_width=6724,
    target_height=5885,
    x_min=None,
    x_max=None,
    y_min=None,
    y_max=None,
    sample_ratio=1.0,
    merge_threshold=0.5,
    percentile=70
)

print('ckpt_path:', args.ckpt_path)
print(f"Dataset: {args.dataset_name}, radius: {args.radius}, max_points: {args.max_points}")

seed_everything(args.seed)

## Generate cell-specific gene embeddings

In [ ]:
cache_path = f"/media/dang/Omics/omics/baseline/cached_spot_data/spot_input_spot_all_{args.dataset_name}_42_{args.gene_pct}_{args.radius}_{args.max_points}.h5"

print("Processing cell-gene pairs for gene embeddings -> AnnData")
adata_result = process_single_gpu_gene_pairs(
    args, args.cell_gene_pairs_path, args.target_dataset, cache_path
)